In [9]:
import sys
sys.path.append('../scripts/')
from puddle_world import *
import itertools 
import collections
%matplotlib widget

In [10]:
class PolicyEvaluator: 
    def __init__(self, widths, goal, puddles, time_interval, sampling_num, \
                 puddle_coef=100.0, lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T):  #puddle_coef追加
        self.pose_min = np.r_[lowerleft, 0]
        self.pose_max = np.r_[upperright, math.pi*2]
        self.widths = widths
        self.goal = goal
        
        self.index_nums = ((self.pose_max - self.pose_min)/self.widths).astype(int)
        nx, ny, nt = self.index_nums
        self.indexes = list(itertools.product(range(nx), range(ny), range(nt)))
        
        self.value_function, self.final_state_flags =  self.init_value_function() 
        self.policy = self.init_policy()
        self.actions = list(set([tuple(self.policy[i]) for i in self.indexes]))
        
        self.state_transition_probs = self.init_state_transition_probs(time_interval, sampling_num)
        self.depths = self.depth_means(puddles, sampling_num)
        
        self.time_interval = time_interval
        self.puddle_coef = puddle_coef
        
    def policy_evaluation_sweep(self):
        max_delta = 0.0
        for index in self.indexes:
            if not self.final_state_flags[index]:
                q = self.action_value(tuple(self.policy[index]), index)
                
                delta = abs(self.value_function[index] - q)
                max_delta = delta if delta > max_delta else max_delta
                
                self.value_function[index] = q
            
        return max_delta
    
    def action_value(self, action, index):
        value = 0.0
        for delta, prob in self.state_transition_probs[(action, index[2])]:
            after = tuple(self.edge_correction(np.array(index).T + delta) )
            reward = - self.time_interval * self.depths[(after[0], after[1])] * self.puddle_coef - self.time_interval
            value += (self.value_function[after] + reward) * prob

        return value
            
    def edge_correction(self, index):
        index[2] = (index[2] + self.index_nums[2])%self.index_nums[2]
                
        return index
        
    def depth_means(self, puddles, sampling_num):
        dx = np.linspace(0, self.widths[0], sampling_num) 
        dy = np.linspace(0, self.widths[1], sampling_num)
        samples = list(itertools.product(dx, dy))
        
        tmp = np.zeros(self.index_nums[0:2])
        for xy in itertools.product(range(self.index_nums[0]), range(self.index_nums[1])):
            for s in samples:
                pose = self.pose_min + self.widths * \
                    np.array([xy[0], xy[1], 0]).T + \
                    np.array([s[0], s[1], 0]).T
                for p in puddles:
                    tmp[xy] += p.depth*p.inside(pose)

            tmp[xy] /= sampling_num**2
                       
        return tmp
    
    def init_state_transition_probs(self, time_interval, sampling_num):
        dx = np.linspace(0.001, self.widths[0]*0.999, sampling_num)
        dy = np.linspace(0.001, self.widths[1]*0.999, sampling_num)
        dt = np.linspace(0.001, self.widths[2]*0.999, sampling_num)
        samples = list(itertools.product(dx, dy, dt))
        
        tmp = {}
        for a in self.actions:
            for i_t in range(self.index_nums[2]):
                transitions = []
                for s in samples:
                    before = np.array([s[0], s[1], s[2] + i_t*self.widths[2]]).T + self.pose_min
                    before_index = np.array([0, 0, i_t]).T
                
                    after = IdealRobot.state_transition(a[0], a[1], time_interval, before)
                    after_index = np.floor((after - self.pose_min)/self.widths).astype(int)
                    
                    transitions.append(after_index - before_index)
                    
                unique, count = np.unique(transitions, axis=0, return_counts=True)
                probs = [c/sampling_num**3 for c in count]
                tmp[a,i_t] = list(zip(unique, probs))
                
        return tmp
        
    def init_policy(self):
        tmp = np.zeros(np.r_[self.index_nums,2])
        for index in self.indexes:
            center = self.pose_min + self.widths*(np.array(index).T + 0.5)
            tmp[index] = PuddleIgnoreAgent.policy(center, self.goal)
            
        return tmp
        
    def init_value_function(self): 
        v = np.empty(self.index_nums)
        f = np.zeros(self.index_nums) 
        
        for index in self.indexes:
            f[index] = self.final_state(np.array(index).T)
            v[index] = self.goal.value if f[index] else -100.0
                
        return v, f
        
    def final_state(self, index):
        x_min, y_min, _ = self.pose_min + self.widths*index
        x_max, y_max, _ = self.pose_min + self.widths*(index + 1)
        
        corners = [[x_min, y_min, _], [x_min, y_max, _], [x_max, y_min, _], [x_max, y_max, _] ]
        return all([self.goal.inside(np.array(c).T) for c in corners ])

In [11]:
import seaborn as sns   

puddles = [Puddle((-2, 0), (0, 2), 0.1), Puddle((-0.5, -2), (2.5, 1), 0.1)] 
pe = PolicyEvaluator(np.array([0.2, 0.2, math.pi/18]).T, Goal(-3,-3), puddles, 0.1, 10)

counter = 0

In [ ]:
delta = 1e100

while delta > 0.01:
    delta = pe.policy_evaluation_sweep()
    counter += 1
    print(counter, delta)

v = pe.value_function[:, :, 18]
sns.heatmap(np.rot90(v), square=False)
plt.show()

1 54.076220000000006
2 53.85750528
3 47.920786948
4 43.165992809471994
5 38.82614599589999
6 34.92590254127054
7 31.559431117143326
8 31.530735627105408
9 30.87633426963437
10 29.785013383303713
11 28.35875085384682
12 26.801810668174326
13 25.88783280697892
14 25.739418781996278
15 25.28198645857328
16 24.578882106661993
17 19.25455310190584
18 17.938698749483393
19 15.776343433340237
20 14.552531262717785
21 13.976243292596749
22 12.865531691213668
23 12.570221372178302
24 11.95071796184613
25 11.259669991523673
26 10.397895995530554
27 9.747425125210732
28 9.451902632534932
29 8.899812415397015
30 8.624448141379325
31 8.340356860166168
32 7.9402594282643335
33 7.6050303727135855
34 7.293399462125443
35 7.033004226681797
36 6.813408506959291
37 6.699075002108387
38 6.5678924515890245
39 6.3802702431715375
40 6.2061096935009274
41 6.044071337759647
42 5.897506602725187
43 5.763942941784833
44 5.6378461325625935
45 5.51871528533561
46 5.406096862065283
47 5.299575974453745
48 5.1987690

<IPython.core.display.Javascript object>

In [ ]:
with open("puddle_ignore_policy.txt", "w") as f:
    for index in pe.indexes:
        p = pe.policy[index]
        f.write("{} {} {} {} {}\n".format(index[0], index[1], index[2], p[0], p[1]))
        
with open("puddle_ignore_values.txt", "w") as f:
    for index in pe.indexes:
        v = pe.value_function[index]
        f.write("{} {} {} {}\n".format(index[0], index[1], index[2],v))